<a href="https://colab.research.google.com/github/imabubakr/Karuna-UG/blob/main/SAM_batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Batch segmentation with text prompts

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/text_prompts_batch.ipynb)
[![image](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/opengeos/segment-geospatial&urlpath=lab/tree/segment-geospatial/docs/examples/text_prompts_batch.ipynb&branch=main)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/text_prompts_batch.ipynb)

This notebook shows how to generate object masks from text prompts with the Segment Anything Model (SAM).

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator.

## Install dependencies

Uncomment and run the following cell to install the required dependencies.

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install segment-geospatial groundingdino-py leafmap localtileserver

In [ ]:
import leafmap
from samgeo import tms_to_geotiff, split_raster
from samgeo.text_sam import LangSAM

## Create an interactive map

In [ ]:
m = leafmap.Map(center=[-22.1278, -51.4430], zoom=17, height="800px")
m.add_basemap("SATELLITE")
m

Map(center=[-22.1278, -51.443], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

## Download a sample image

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [ ]:
bbox = m.user_roi_bounds()
if bbox is None:
    bbox = [-51.4494, -22.1307, -51.4371, -22.1244]

In [ ]:
image = '/content/drive/MyDrive/MRC/satellite.tif'
tms_to_geotiff(output=image, bbox=bbox, zoom=19, source="Satellite", overwrite=True)

In [ ]:
image = '/content/drive/MyDrive/MRC/satellite.tif'

You can also use your own image. Uncomment and run the following cell to use your own image.

Display the downloaded image on the map.

In [ ]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

NameError: name 'm' is not defined

## Split the image into tiles

In [ ]:
split_raster(image, out_dir="tiles", tile_size=(500, 500), overlap=0)

## Initialize LangSAM class

The initialization of the LangSAM class might take a few minutes. The initialization downloads the model weights and sets up the model for inference.

In [ ]:
sam = LangSAM()

final text_encoder_type: bert-base-uncased


## Specify text prompts

In [ ]:
text_prompt = "tree"

## Segment images

Part of the model prediction includes setting appropriate thresholds for object detection and text association with the detected objects. These threshold values range from 0 to 1 and are set while calling the predict method of the LangSAM class.

`box_threshold`: This value is used for object detection in the image. A higher value makes the model more selective, identifying only the most confident object instances, leading to fewer overall detections. A lower value, conversely, makes the model more tolerant, leading to increased detections, including potentially less confident ones.

`text_threshold`: This value is used to associate the detected objects with the provided text prompt. A higher value requires a stronger association between the object and the text prompt, leading to more precise but potentially fewer associations. A lower value allows for looser associations, which could increase the number of associations but also introduce less precise matches.

Remember to test different threshold values on your specific data. The optimal threshold can vary depending on the quality and nature of your images, as well as the specificity of your text prompts. Make sure to choose a balance that suits your requirements, whether that's precision or recall.

In [ ]:
sam.predict_batch(
    images="tiles",
    out_dir="masks",
    text_prompt=text_prompt,
    box_threshold=0.34,
    text_threshold=0.34,
    mask_multiplier=255,
    dtype="uint8",
    merge=True,
    verbose=True,
)

Processing image 001 of 434: tiles/tile_0_0.tif...
Processing image 002 of 434: tiles/tile_0_1.tif...
Processing image 003 of 434: tiles/tile_0_10.tif...
No objects found in the image.
Processing image 004 of 434: tiles/tile_0_11.tif...
No objects found in the image.
Processing image 005 of 434: tiles/tile_0_12.tif...
No objects found in the image.
Processing image 006 of 434: tiles/tile_0_13.tif...
Processing image 007 of 434: tiles/tile_0_14.tif...
No objects found in the image.
Processing image 008 of 434: tiles/tile_0_15.tif...
Processing image 009 of 434: tiles/tile_0_16.tif...
No objects found in the image.
Processing image 010 of 434: tiles/tile_0_17.tif...
No objects found in the image.
Processing image 011 of 434: tiles/tile_0_18.tif...
Processing image 012 of 434: tiles/tile_0_19.tif...
Processing image 013 of 434: tiles/tile_0_2.tif...
Processing image 014 of 434: tiles/tile_0_20.tif...
No objects found in the image.
Processing image 015 of 434: tiles/tile_0_21.tif...
No obj

## Visualize the results

In [ ]:
m.add_raster("masks/merged.tif", cmap="viridis", nodata=0, layer_name="Mask")
m.add_layer_manager()
m

NameError: name 'm' is not defined

In [ ]:
sam.raster_to_vector("masks/merged.tif", "/content/drive/MyDrive/MRC/predict/tree-canopies.tif")

![](https://i.imgur.com/JUhNkm6.png)